### Gerekli Kütüphanelerin Yüklenmesi

In [19]:
import os 
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
import time
from sklearn.model_selection import train_test_split
import cv2

In [20]:
print(os.getcwd())

c:\Users\LENOVO\Desktop\ai_recycling_machine


In [21]:
os.chdir(r"c:\Users\LENOVO\Desktop\ai_recycling_machine")

### Veri Seti ve Etiketlerin Yüklenmesi

In [22]:
# Veriyi ve etiket dosyasını yükleme
image_directory = "dataset\dataset-resized"
labels_train_file = r"C:\Users\LENOVO\Desktop\ai_recycling_machine\labels\one-indexed-files-notrash_train.txt"

labels_test_file = r"C:\Users\LENOVO\Desktop\ai_recycling_machine\labels\one-indexed-files-notrash_test.txt"

labels_val_file = r"C:\Users\LENOVO\Desktop\ai_recycling_machine\labels\one-indexed-files-notrash_val.txt"

In [23]:
# Dosya yolunu vererek etiketleri yükleyen bir fonksiyon tanımlıyoruz
def load_labels(file_path):
    labels = {}
    with open(file_path, 'r') as f:
        for line in f.readlines():
            filename, label = line.strip().split()
            labels[filename] = int(label)
    return labels

# Etiketleri yükleme
labels_train = load_labels(labels_train_file)
labels_test = load_labels(labels_test_file)
labels_val = load_labels(labels_val_file)

# Yüklenen etiketleri kontrol etmek için
print("Eğitim Etiketleri:", labels_train)
print("Test Etiketleri:", labels_test)
print("Doğrulama Etiketleri:", labels_val)

Eğitim Etiketleri: {'cardboard202.jpg': 3, 'paper472.jpg': 2, 'paper522.jpg': 2, 'glass189.jpg': 1, 'glass325.jpg': 1, 'metal63.jpg': 5, 'cardboard61.jpg': 3, 'trash50.jpg': 4, 'paper305.jpg': 2, 'paper199.jpg': 2, 'paper539.jpg': 2, 'plastic262.jpg': 4, 'plastic210.jpg': 4, 'glass489.jpg': 1, 'cardboard369.jpg': 3, 'glass80.jpg': 1, 'glass97.jpg': 1, 'plastic275.jpg': 4, 'glass272.jpg': 1, 'glass380.jpg': 1, 'paper96.jpg': 2, 'paper17.jpg': 2, 'glass301.jpg': 1, 'paper523.jpg': 2, 'glass66.jpg': 1, 'plastic344.jpg': 4, 'paper401.jpg': 2, 'trash88.jpg': 6, 'paper414.jpg': 2, 'metal25.jpg': 5, 'glass241.jpg': 1, 'metal231.jpg': 5, 'metal329.jpg': 5, 'plastic140.jpg': 4, 'paper252.jpg': 2, 'paper205.jpg': 2, 'paper107.jpg': 2, 'trash80.jpg': 3, 'plastic271.jpg': 4, 'cardboard62.jpg': 3, 'paper410.jpg': 2, 'metal95.jpg': 5, 'paper256.jpg': 3, 'glass282.jpg': 1, 'plastic28.jpg': 4, 'paper338.jpg': 2, 'metal404.jpg': 5, 'paper33.jpg': 2, 'paper131.jpg': 2, 'metal227.jpg': 5, 'glass413.jpg':

In [24]:
# Veriyi hazırlama
def load_images(labels):
    images = []
    labels_list = []
    for filename, label in labels.items():
        # Etiketlere göre alt klasör adını belirleme
        if label == 1:
            folder = "glass"
        elif label == 2:
            folder = "paper"
        elif label == 3:
            folder = "cardboard"
        elif label == 4:
            folder = "plastic"
        elif label == 5:
            folder = "metal"
        elif label == 6:
            folder = "trash"
        
        # Tam dosya yolunu oluşturma
        img_path = os.path.join(image_directory, folder, filename)
        
        img = cv2.imread(img_path)
        if img is None:
            print(f"Resim yüklenemedi: {img_path}")
            continue  # Bu görüntüyü atlayarak sonraki görüntüye geç
        
        img = cv2.resize(img, (224, 224))  # YOLO modelleri için uygun boyut
        images.append(img)
        labels_list.append(label)
        
    return np.array(images, dtype='float32') / 255.0, np.array(labels_list)



In [25]:
# Eğitim, test ve doğrulama verilerini yükleme
X_train, y_train = load_images(labels_train)
X_test, y_test = load_images(labels_test)
X_val, y_val = load_images(labels_val)


Resim yüklenemedi: dataset\dataset-resized\cardboard\trash80.jpg
Resim yüklenemedi: dataset\dataset-resized\trash\trash2.jpg
Resim yüklenemedi: dataset\dataset-resized\cardboard\trash22.jpg


### Modelin Geliştirilmesi

In [26]:
from tensorflow.keras.applications import MobileNetV2

# Önceden eğitilmiş MobileNetV2 modelini yükleyin ve yalnızca son katmanları eğitin
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Tüm katmanları donduruyoruz

model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(6, activation='softmax')
])

In [27]:
# Etiketleri sıfır bazlı yapmak
y_train -= 1
y_test -= 1
y_val -= 1

In [28]:
log_dir = "logs/fit/" + time.strftime("%Y%m%d-%H%M%S")

# TensorBoard callback
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [29]:
# Modeli derleme
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Modeli eğitme, TensorBoard ve EarlyStopping callback'lerini ekleyin
model.fit(X_train, y_train, epochs=30, validation_data=(X_val, y_val), callbacks=[tensorboard_callback, early_stopping])


# Modeli değerlendirme
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test doğruluğu: {accuracy * 100:.2f}%')

Epoch 1/30
62/62 [==============================] - 68s 1s/step - loss: 1.1838 - accuracy: 0.5464 - val_loss: 0.7345 - val_accuracy: 0.7256
Epoch 2/30
62/62 [==============================] - 61s 989ms/step - loss: 0.7528 - accuracy: 0.7235 - val_loss: 0.6520 - val_accuracy: 0.7591
Epoch 3/30
62/62 [==============================] - 62s 999ms/step - loss: 0.5760 - accuracy: 0.7889 - val_loss: 0.5491 - val_accuracy: 0.8140
Epoch 4/30
62/62 [==============================] - 58s 933ms/step - loss: 0.5090 - accuracy: 0.8113 - val_loss: 0.5468 - val_accuracy: 0.7957
Epoch 5/30
62/62 [==============================] - 58s 932ms/step - loss: 0.4081 - accuracy: 0.8559 - val_loss: 0.5124 - val_accuracy: 0.8232
Epoch 6/30
62/62 [==============================] - 63s 1s/step - loss: 0.3567 - accuracy: 0.8681 - val_loss: 0.5242 - val_accuracy: 0.8079
Epoch 7/30
62/62 [==============================] - 56s 896ms/step - loss: 0.3206 - accuracy: 0.8879 - val_loss: 0.4867 - val_accuracy: 0.8232
Epoch

In [30]:
# Modeli kaydetme
model.save('model/model.keras')